In [ ]:
# Import our dependencies# Import 
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model
import numpy as np
from keras.models import load_model, model_from_config
import os

# Initialize session
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
sess = tf.Session(config=config)
K.set_session(sess)

In [2]:
# Now instantiate the elmo m1odel
elmo_model_path = "./elmo_model/"

if not os.path.isdir(elmo_model_path):
    elmo_model_path = "https://tfhub.dev/google/elmo/2"

elmo_model = hub.Module(elmo_model_path, trainable=False)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


In [3]:
# Build our model# Build 

# We create a function to integrate the tensorflow model with a Keras model
# This requires explicitly casting the tensor to a string, because of a Keras quirk
def ElmoEmbedding(x):
    return elmo_model(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [4]:
# load data
import pandas as pd
dtypes_train = {"id":np.int64, "text":str, "author":str, "title":str, "label":np.int64}
dtypes_test = {"id":np.int64, "text":str, "author":str, "title":str,}

In [5]:
train_df = pd.read_csv("data/train.csv", dtype=dtypes_train)
train_df = train_df.dropna()
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
X_train = train_df.drop(['label', 'id', 'author'], axis=1).values
Y_train = train_df['label'].values

print("Train DIMS \nX dims: {} Y dims: {}".format(X_train.shape, Y_train.shape))

Train DIMS 
X dims: (18285, 2) Y dims: (18285,)


In [7]:
test_df = pd.read_csv("data/test.csv")
test_df = test_df.dropna()
test_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
6,20806,Pelosi Calls for FBI Investigation to Find Out...,Pam Key,"Sunday on NBC’s “Meet the Press,” House Minori..."


In [8]:
X_test = test_df.drop(['id', 'author'], axis=1).values

In [9]:
print("Test DIMS \nX dims: {}".format(X_test.shape))

Test DIMS 
X dims: (4575, 2)


In [10]:
for i in range(len(X_train)):
    X_train[i][0] = np.array(X_train[i][0].split())
    X_train[i][1] = np.array(X_train[i][1].split())
    
for i in range(len(X_test)): 
    X_test[i][0] = np.array(X_test[i][0].split())
    X_test[i][1] = np.array(X_test[i][1].split())

In [10]:
rnn_dim = 10
dense_dim = 128
output_dim = 1
batch_size = 16
max_length = 75
from keras.layers import *

In [11]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding_text = Lambda(ElmoEmbedding, output_shape=(1024,))(input_text)
# embedding_text = Reshape((1024, 1))(embedding_text)

# input_title = Input(shape=(1,), dtype=tf.string)
# embedding_title = Lambda(ElmoEmbedding, output_shape=(max_length, 512))(input_title)
# embedding_title = Reshape((1024, 1))(embedding_title)

# text_encoding = Bidirectional(LSTM(rnn_dim, return_sequences=True))(embedding_text)
# title_encoding = Bidirectional(LSTM(rnn_dim, return_sequences=True))(embedding_title)

# merged_layer = merge.Concatenate()([text_encoding, title_encoding])

# falten_layer = Flatten()(embedding_text)

# dense = (Dense(dense_dim, activation='relu'))(falten_layer)
preds = (Dense(output_dim, activation='sigmoid'))(embedding_text)

model = Model(inputs=[input_text], outputs=preds)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1025      
Total params: 1,025
Trainable params: 1,025
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train[:10,1], Y_train[:10], 
          epochs=10)

Epoch 1/10
10/10 [==============================] - 118s 12s/step - loss: 0.7016 - acc: 0.3000
Epoch 2/10
10/10 [==============================] - 117s 12s/step - loss: 0.6831 - acc: 0.6000
Epoch 3/10


In [2]:
from allennlp.modules.elmo import Elmo, batch_to_ids

weights_path = "elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5"
options_path = "elmo_small_options.json"

model = Elmo(weight_file=weights_path, options_file=options_path, num_output_representations=1)